## Import Libraries 

In [1]:
import sys
sys.path.append('/paths_generator.py')
import paths_generator as pg
import random as rand
import math as m
import numpy as np

## Simulated Annealing Function

In [1]:
def simulated_annealing (init_st, cost_func, max_it, max_T, cooling_rate):   #cooling_rate howa li ki khli T decreasa
  current_st = init_st
  best_st = init_st
  T = max_T

  for i in range(max_it):
    if T <= 0:   
      break

    new_st = current_st.get_neighbor()   # Get a random neighbor of the current state
    current_C = cost_func(current_st)
    new_C = cost_func(new_C)

    if new_C < current_C:
      current_st = new_st # Accept the new state as the current state
      if new_C < cost_funct(best_st):
        best_state = new_state  # Update the best state if the new state has a minimal cost
    
    else: 
      proba = m.exp(- (current_C - new_C) / T)
      if rand.random() < proba:
        current_st = new_st
        T *= cooling_rate # Cool down the temperature for the next iteration

  return best_st

## Define Ctes based on the possible paths

In [ ]:
m = 1
n = 1
e_max = 1  # numbers
e_i = 1

## Optimization Problem

In [3]:
def objective(x):  #m, n depends on the path: temperary
  for i in range(1, m): # m: present the number of places where we will put our edge
    for j in range(1, n): # n: present the number of places that we will prevet water flow from thanks to the edge
      np.sum(x[i][j]*(e_max - e_i))